In [1]:
from greedy_mcmc_attack import *
from data_collection.recordings import *
import sys
sys.path.append("../../")
from src.models.gcn import *
from src.models.gat import *
from src.models.gsage import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

trials = 5

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
edges_to_add = None

In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [6]:
model_save_path = "../models/cora_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges_full_run.pth"
# list_save_path = "../attacks/cora_gcn_edges.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [7]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [8]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.806
Initial Loss: 0.655889093875885


In [9]:
G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.20)

In [10]:
full_metattack_acc, full_metattack_loss, full_metattack_itrs = [], [], []
for _ in range(trials):
    acc, loss, itrs = two_phase_attack_greedy(data, train, model, 0.0, edges_to_add, constant_fn, device, is_reversed=False, verbose=True)
    full_metattack_acc.append(acc)
    full_metattack_loss.append(loss)
    full_metattack_itrs.append(itrs)

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6076424717903137
Surrogate acc on unlabled data: 0.805
Attack loss: 0.2829137444496155
Surrogate loss on unlabled data: 0.7039027810096741
Surrogate acc on unlabled data: 0.781
Attack loss: 0.4154606759548187
Surrogate loss on unlabled data: 0.9057208299636841
Surrogate acc on unlabled data: 0.736
Attack loss: 0.6127167344093323
Surrogate loss on unlabled data: 0.8646247982978821
Surrogate acc on unlabled data: 0.725
Attack loss: 0.6141096353530884
Surrogate loss on unlabled data: 1.0203737020492554
Surrogate acc on unlabled data: 0.682
Attack loss: 0.7593544721603394
Surrogate loss on unlabled data: 1.1040681600570679
Surrogate acc on unlabled data: 0.645
Attack loss: 0.867948591709137
Surrogate loss on unlabled data: 1.1518312692642212
Surrogate acc on unlabled data: 0.642
Attack loss: 0.9360206723213196
Surrogate loss on unlabled data: 1.2556995153427124
Surrogate acc on unlabled data: 0.635
Attack loss: 1.0342254638671875
Surrogate loss on unlable

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.5966231226921082
Surrogate acc on unlabled data: 0.808
Attack loss: 0.28638315200805664
Surrogate loss on unlabled data: 0.7257119417190552
Surrogate acc on unlabled data: 0.78
Attack loss: 0.4335567057132721
Surrogate loss on unlabled data: 0.8151730298995972
Surrogate acc on unlabled data: 0.751
Attack loss: 0.519555926322937
Surrogate loss on unlabled data: 0.9013957381248474
Surrogate acc on unlabled data: 0.741
Attack loss: 0.630111813545227
Surrogate loss on unlabled data: 0.987515389919281
Surrogate acc on unlabled data: 0.714
Attack loss: 0.7207518815994263
Surrogate loss on unlabled data: 1.046435832977295
Surrogate acc on unlabled data: 0.6910000000000001
Attack loss: 0.8036395907402039
Surrogate loss on unlabled data: 1.1538233757019043
Surrogate acc on unlabled data: 0.659
Attack loss: 0.9270074367523193
Surrogate loss on unlabled data: 1.2712140083312988
Surrogate acc on unlabled data: 0.631
Attack loss: 1.0364372730255127
Surrogate loss 

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

...

Surrogate loss on unlabled data: 0.588080644607544
Surrogate acc on unlabled data: 0.799
Attack loss: 0.27092865109443665
Surrogate loss on unlabled data: 0.678314208984375
Surrogate acc on unlabled data: 0.793
Attack loss: 0.42234259843826294
Surrogate loss on unlabled data: 0.7793538570404053
Surrogate acc on unlabled data: 0.758
Attack loss: 0.4994984269142151
Surrogate loss on unlabled data: 0.8907495141029358
Surrogate acc on unlabled data: 0.727
Attack loss: 0.6242908239364624
Surrogate loss on unlabled data: 0.9378762245178223
Surrogate acc on unlabled data: 0.709
Attack loss: 0.680306613445282
Surrogate loss on unlabled data: 1.0054835081100464
Surrogate acc on unlabled data: 0.678
Attack loss: 0.8057897686958313
Surrogate loss on unlabled data: 1.219265103340149
Surrogate acc on unlabled data: 0.63
Attack loss: 0.9906026124954224
Surrogate loss on unlabled data: 1.2569252252578735
Surrogate acc on unlabled data: 0.627
Attack loss: 1.017806887626648
Surrogate loss on unlabled d

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.5841187834739685
Surrogate acc on unlabled data: 0.811
Attack loss: 0.2763432264328003
Surrogate loss on unlabled data: 0.7464727759361267
Surrogate acc on unlabled data: 0.771
Attack loss: 0.4382922351360321
Surrogate loss on unlabled data: 0.7981606721878052
Surrogate acc on unlabled data: 0.766
Attack loss: 0.5116552114486694
Surrogate loss on unlabled data: 0.8693077564239502
Surrogate acc on unlabled data: 0.746
Attack loss: 0.6213132739067078
Surrogate loss on unlabled data: 0.8987817764282227
Surrogate acc on unlabled data: 0.724
Attack loss: 0.6536430716514587
Surrogate loss on unlabled data: 1.003635048866272
Surrogate acc on unlabled data: 0.6950000000000001
Attack loss: 0.7867962718009949
Surrogate loss on unlabled data: 1.0678904056549072
Surrogate acc on unlabled data: 0.671
Attack loss: 0.8456593155860901
Surrogate loss on unlabled data: 1.2195422649383545
Surrogate acc on unlabled data: 0.648
Attack loss: 0.983550488948822
Surrogate los

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6064630746841431
Surrogate acc on unlabled data: 0.808
Attack loss: 0.26813405752182007
Surrogate loss on unlabled data: 0.7217533588409424
Surrogate acc on unlabled data: 0.78
Attack loss: 0.4276944696903229
Surrogate loss on unlabled data: 0.7432616949081421
Surrogate acc on unlabled data: 0.772
Attack loss: 0.46294698119163513
Surrogate loss on unlabled data: 0.877964198589325
Surrogate acc on unlabled data: 0.743
Attack loss: 0.5650919079780579
Surrogate loss on unlabled data: 0.9882364273071289
Surrogate acc on unlabled data: 0.728
Attack loss: 0.6847435235977173
Surrogate loss on unlabled data: 1.0713598728179932
Surrogate acc on unlabled data: 0.7000000000000001
Attack loss: 0.7762593030929565
Surrogate loss on unlabled data: 1.1251769065856934
Surrogate acc on unlabled data: 0.679
Attack loss: 0.829727053642273
Surrogate loss on unlabled data: 1.1946247816085815
Surrogate acc on unlabled data: 0.671
Attack loss: 0.9215747117996216
Surrogate lo

In [11]:
edges_to_add

[(115, 2644),
 (57, 2602),
 (13, 2603),
 (49, 1938),
 (79, 2513),
 (79, 1582),
 (79, 2697),
 (79, 1446),
 (79, 2633),
 (79, 2568),
 (79, 1005),
 (79, 2142),
 (54, 2529),
 (23, 2258),
 (23, 422),
 (23, 2149),
 (86, 2300),
 (60, 2699),
 (49, 1056),
 (23, 131),
 (131, 322),
 (23, 2698),
 (23, 1364),
 (23, 692),
 (23, 2700),
 (29, 2482),
 (38, 2438),
 (96, 2573),
 (654, 2683),
 (50, 2299),
 (76, 1796),
 (23, 1591),
 (23, 1811),
 (7, 2417),
 (21, 2416),
 (131, 2411),
 (136, 2410),
 (8, 19),
 (7, 63),
 (7, 1643),
 (63, 167),
 (23, 2559),
 (7, 1018),
 (23, 2693),
 (23, 2635),
 (1032, 2298),
 (23, 1615),
 (1, 7),
 (7, 2549),
 (23, 2626),
 (23, 1014),
 (82, 2429),
 (23, 1018),
 (23, 2703),
 (131, 422),
 (23, 311),
 (23, 2605),
 (0, 2617),
 (7, 2694),
 (7, 2548),
 (27, 2619),
 (66, 2437),
 (7, 2264),
 (23, 1164),
 (21, 2575),
 (23, 79),
 (79, 313),
 (23, 2477),
 (23, 2264),
 (23, 1860),
 (79, 625),
 (79, 2551),
 (12, 250),
 (57, 2618),
 (25, 1626),
 (23, 1447),
 (131, 1200),
 (131, 2701),
 (131,

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

# regular, secondary attack, not reversed

In [12]:
# split_dic_acc

In [13]:
splits = [0.0, 0.5]

split_dic_acc = defaultdict(lambda: defaultdict(list))
split_dic_loss = defaultdict(lambda: defaultdict(list))
itrs_one = defaultdict(lambda: defaultdict(int))

In [14]:
# data

In [15]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = two_phase_attack_greedy(data, train, model, s, edges_to_add, constant_fn, device, is_reversed=False, verbose=True)

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.5954928398132324
Surrogate acc on unlabled data: 0.805
Attack loss: 0.29173192381858826
Surrogate loss on unlabled data: 0.662014901638031
Surrogate acc on unlabled data: 0.791
Attack loss: 0.37607043981552124
Surrogate loss on unlabled data: 0.7193514108657837
Surrogate acc on unlabled data: 0.778
Attack loss: 0.4983190894126892
Surrogate loss on unlabled data: 0.8468896746635437
Surrogate acc on unlabled data: 0.746
Attack loss: 0.6149662137031555
Surrogate loss on unlabled data: 0.9451941847801208
Surrogate acc on unlabled data: 0.734
Attack loss: 0.7308712601661682
Surrogate loss on unlabled data: 0.9687224626541138
Surrogate acc on unlabled data: 0.718
Attack loss: 0.7648036479949951
Surrogate loss on unlabled data: 0.9872097373008728
Surrogate acc on unlabled data: 0.7030000000000001
Attack loss: 0.8058753609657288
Surrogate loss on unlabled data: 1.1350551843643188
Surrogate acc on unlabled data: 0.654
Attack loss: 0.9530410766601562
Surrogate 

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6739389896392822
Surrogate acc on unlabled data: 0.788
Attack loss: 0.3768647313117981
Surrogate loss on unlabled data: 0.870611310005188
Surrogate acc on unlabled data: 0.724
Attack loss: 0.5758737921714783
Surrogate loss on unlabled data: 0.9718067049980164
Surrogate acc on unlabled data: 0.684
Attack loss: 0.6793938279151917
Surrogate loss on unlabled data: 1.1272099018096924
Surrogate acc on unlabled data: 0.649
Attack loss: 0.84462970495224
Surrogate loss on unlabled data: 1.1446924209594727
Surrogate acc on unlabled data: 0.631
Attack loss: 0.8731677532196045
Surrogate loss on unlabled data: 1.2884408235549927
Surrogate acc on unlabled data: 0.6
Attack loss: 1.0479356050491333
Surrogate loss on unlabled data: 1.3806730508804321
Surrogate acc on unlabled data: 0.604
Attack loss: 1.1293094158172607
Surrogate loss on unlabled data: 1.5114206075668335
Surrogate acc on unlabled data: 0.5670000000000001
Attack loss: 1.2818350791931152
Surrogate loss o

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

...

Surrogate loss on unlabled data: 0.7117354273796082
Surrogate acc on unlabled data: 0.76
Attack loss: 0.4322364926338196
Surrogate loss on unlabled data: 0.8242855668067932
Surrogate acc on unlabled data: 0.741
Attack loss: 0.5139445662498474
Surrogate loss on unlabled data: 0.9733035564422607
Surrogate acc on unlabled data: 0.713
Attack loss: 0.6861823797225952
Surrogate loss on unlabled data: 1.1097897291183472
Surrogate acc on unlabled data: 0.673
Attack loss: 0.8500232696533203
Surrogate loss on unlabled data: 1.2262483835220337
Surrogate acc on unlabled data: 0.629
Attack loss: 0.9856588840484619
Surrogate loss on unlabled data: 1.249044418334961
Surrogate acc on unlabled data: 0.615
Attack loss: 1.0141465663909912
Surrogate loss on unlabled data: 1.3771188259124756
Surrogate acc on unlabled data: 0.59
Attack loss: 1.166233777999878
Surrogate loss on unlabled data: 1.4917290210723877
Surrogate acc on unlabled data: 0.5640000000000001
Attack loss: 1.2748546600341797
Surrogate loss 

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6061086654663086
Surrogate acc on unlabled data: 0.807
Attack loss: 0.2808392643928528
Surrogate loss on unlabled data: 0.7039760947227478
Surrogate acc on unlabled data: 0.788
Attack loss: 0.41243216395378113
Surrogate loss on unlabled data: 0.8117767572402954
Surrogate acc on unlabled data: 0.743
Attack loss: 0.5193340182304382
Surrogate loss on unlabled data: 0.935574471950531
Surrogate acc on unlabled data: 0.728
Attack loss: 0.6488599181175232
Surrogate loss on unlabled data: 0.9387117624282837
Surrogate acc on unlabled data: 0.707
Attack loss: 0.7117989659309387
Surrogate loss on unlabled data: 1.0935965776443481
Surrogate acc on unlabled data: 0.6980000000000001
Attack loss: 0.834975004196167
Surrogate loss on unlabled data: 1.269824504852295
Surrogate acc on unlabled data: 0.673
Attack loss: 0.9937596321105957
Surrogate loss on unlabled data: 1.2939088344573975
Surrogate acc on unlabled data: 0.629
Attack loss: 1.0665282011032104
Surrogate los

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.7076516151428223
Surrogate acc on unlabled data: 0.785
Attack loss: 0.37065744400024414
Surrogate loss on unlabled data: 0.818057119846344
Surrogate acc on unlabled data: 0.754
Attack loss: 0.537472128868103
Surrogate loss on unlabled data: 0.871641993522644
Surrogate acc on unlabled data: 0.749
Attack loss: 0.5926817655563354
Surrogate loss on unlabled data: 0.99848473072052
Surrogate acc on unlabled data: 0.6990000000000001
Attack loss: 0.7377284169197083
Surrogate loss on unlabled data: 1.1576411724090576
Surrogate acc on unlabled data: 0.638
Attack loss: 0.9015117883682251
Surrogate loss on unlabled data: 1.2367134094238281
Surrogate acc on unlabled data: 0.609
Attack loss: 0.9956449270248413
Surrogate loss on unlabled data: 1.3397332429885864
Surrogate acc on unlabled data: 0.586
Attack loss: 1.116973876953125
Surrogate loss on unlabled data: 1.5040028095245361
Surrogate acc on unlabled data: 0.5710000000000001
Attack loss: 1.2777111530303955
Sur

{0.0: defaultdict(list,
             {0: [0.655889093875885,
               0.652009129524231,
               0.6547946929931641,
               0.6580879092216492,
               0.6587175726890564,
               0.6581577062606812,
               0.6580770611763,
               0.658064067363739,
               0.6603716015815735,
               0.6601581573486328,
               0.6601480841636658,
               0.6601428985595703,
               0.6600430011749268,
               0.6607937216758728,
               0.6608331203460693,
               0.6649040579795837,
               0.6649230718612671,
               0.6649867296218872,
               0.6649804711341858,
               0.6709028482437134,
               0.670391857624054,
               0.6703411340713501,
               0.6742852926254272,
               0.6734521389007568,
               0.6739587187767029,
               0.6739966869354248,
               0.6792680621147156,
               0.6792660355567932,


In [22]:
record1 = Recording(dict(split_dic_loss), dict(split_dic_acc), dict(itrs_one), Model.GCN, Dataset.CORA, AcceptFn.CONSTANT, SelectFn.NONE, Reverse.ATTACKED)
record1.save(f'data_collection/{"greedy"}_{"gcn"}_{"cora"}_{"constant"}_{"attacked"}_results.pkl')

In [ ]:
split_dic_loss

In [ ]:
# plot_results({0: split_dic_acc[0.0][0], 0.5: split_dic_acc[0.5][0]}, ptb_rate, "Greedy", "", "constant", "accuracy", "regular")

In [ ]:
# plot_results({0: split_dic_loss[0.0][0], 0.5: split_dic_loss[0.5][0]}, ptb_rate, "Greedy", "", "constant", "loss", "regular")

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

In [23]:
split_dic_decreasing_acc = defaultdict(lambda: defaultdict(list))
itrs_two = defaultdict(lambda: defaultdict(int))
split_dic_decreasing_loss = defaultdict(lambda: defaultdict(list))

In [24]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = two_phase_attack_greedy(data, train, model, s, edges_to_add, increasing_fn, device, is_reversed=False, verbose=True)

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6017633080482483
Surrogate acc on unlabled data: 0.808
Attack loss: 0.2876338064670563
Surrogate loss on unlabled data: 0.7149999737739563
Surrogate acc on unlabled data: 0.782
Attack loss: 0.39649757742881775
Surrogate loss on unlabled data: 0.7200514078140259
Surrogate acc on unlabled data: 0.772
Attack loss: 0.4504481256008148
Surrogate loss on unlabled data: 0.8287292718887329
Surrogate acc on unlabled data: 0.745
Attack loss: 0.5915143489837646
Surrogate loss on unlabled data: 0.8858562707901001
Surrogate acc on unlabled data: 0.723
Attack loss: 0.6445236206054688
Surrogate loss on unlabled data: 0.9854551553726196
Surrogate acc on unlabled data: 0.716
Attack loss: 0.7434565424919128
Surrogate loss on unlabled data: 1.0203778743743896
Surrogate acc on unlabled data: 0.6920000000000001
Attack loss: 0.7941333055496216
Surrogate loss on unlabled data: 1.1219865083694458
Surrogate acc on unlabled data: 0.668
Attack loss: 0.8877072334289551
Surrogate 

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6917418241500854
Surrogate acc on unlabled data: 0.774
Attack loss: 0.3768748641014099
Surrogate loss on unlabled data: 0.8517364859580994
Surrogate acc on unlabled data: 0.731
Attack loss: 0.5369574427604675
Surrogate loss on unlabled data: 0.9150480031967163
Surrogate acc on unlabled data: 0.709
Attack loss: 0.6614792346954346
Surrogate loss on unlabled data: 1.0422536134719849
Surrogate acc on unlabled data: 0.666
Attack loss: 0.8028838038444519
Surrogate loss on unlabled data: 1.1044511795043945
Surrogate acc on unlabled data: 0.639
Attack loss: 0.9041872620582581
Surrogate loss on unlabled data: 1.1273350715637207
Surrogate acc on unlabled data: 0.621
Attack loss: 0.9457533359527588
Surrogate loss on unlabled data: 1.4334564208984375
Surrogate acc on unlabled data: 0.5750000000000001
Attack loss: 1.222917914390564
Surrogate loss on unlabled data: 1.4849921464920044
Surrogate acc on unlabled data: 0.558
Attack loss: 1.309667706489563
Surrogate los

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

...

Surrogate loss on unlabled data: 0.6810005307197571
Surrogate acc on unlabled data: 0.783
Attack loss: 0.37597891688346863
Surrogate loss on unlabled data: 0.8365859389305115
Surrogate acc on unlabled data: 0.732
Attack loss: 0.5645199418067932
Surrogate loss on unlabled data: 0.9079881906509399
Surrogate acc on unlabled data: 0.7000000000000001
Attack loss: 0.6845757961273193
Surrogate loss on unlabled data: 1.0732662677764893
Surrogate acc on unlabled data: 0.652
Attack loss: 0.859740138053894
Surrogate loss on unlabled data: 1.1401816606521606
Surrogate acc on unlabled data: 0.633
Attack loss: 0.9372100830078125
Surrogate loss on unlabled data: 1.2978034019470215
Surrogate acc on unlabled data: 0.605
Attack loss: 1.096501111984253
Surrogate loss on unlabled data: 1.3608684539794922
Surrogate acc on unlabled data: 0.578
Attack loss: 1.1436562538146973
Surrogate loss on unlabled data: 1.4115006923675537
Surrogate acc on unlabled data: 0.5640000000000001
Attack loss: 1.1833384037017822

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6091565489768982
Surrogate acc on unlabled data: 0.808
Attack loss: 0.2758527398109436
Surrogate loss on unlabled data: 0.6814154386520386
Surrogate acc on unlabled data: 0.785
Attack loss: 0.4064343273639679
Surrogate loss on unlabled data: 0.7472244501113892
Surrogate acc on unlabled data: 0.768
Attack loss: 0.48371198773384094
Surrogate loss on unlabled data: 0.8693332672119141
Surrogate acc on unlabled data: 0.735
Attack loss: 0.6039717793464661
Surrogate loss on unlabled data: 0.9620224237442017
Surrogate acc on unlabled data: 0.7030000000000001
Attack loss: 0.6917778849601746
Surrogate loss on unlabled data: 1.0164278745651245
Surrogate acc on unlabled data: 0.6940000000000001
Attack loss: 0.7536566257476807
Surrogate loss on unlabled data: 1.0434074401855469
Surrogate acc on unlabled data: 0.68
Attack loss: 0.8037341237068176
Surrogate loss on unlabled data: 1.159009575843811
Surrogate acc on unlabled data: 0.636
Attack loss: 0.9504067897796631

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.655655026435852
Surrogate acc on unlabled data: 0.784
Attack loss: 0.36225372552871704
Surrogate loss on unlabled data: 0.8164001703262329
Surrogate acc on unlabled data: 0.745
Attack loss: 0.543961226940155
Surrogate loss on unlabled data: 0.8630631566047668
Surrogate acc on unlabled data: 0.722
Attack loss: 0.6157232522964478
Surrogate loss on unlabled data: 0.9820384383201599
Surrogate acc on unlabled data: 0.671
Attack loss: 0.7380350828170776
Surrogate loss on unlabled data: 1.0873571634292603
Surrogate acc on unlabled data: 0.647
Attack loss: 0.8636791706085205
Surrogate loss on unlabled data: 1.2516587972640991
Surrogate acc on unlabled data: 0.62
Attack loss: 1.043624758720398
Surrogate loss on unlabled data: 1.1419705152511597
Surrogate acc on unlabled data: 0.618
Attack loss: 0.9689227938652039
Surrogate loss on unlabled data: 1.4038783311843872
Surrogate acc on unlabled data: 0.596
Attack loss: 1.2186542749404907
Surrogate loss on unlabled 

In [25]:
split_dic_decreasing_loss

defaultdict(<function __main__.<lambda>()>,
            {0.0: defaultdict(list,
                         {0: [0.655889093875885,
                           0.652009129524231,
                           0.6547946929931641,
                           0.6580879092216492,
                           0.6587175726890564,
                           0.6581577062606812,
                           0.6580770611763,
                           0.658064067363739,
                           0.6603716015815735,
                           0.6601581573486328,
                           0.6601480841636658,
                           0.6601428985595703,
                           0.6600430011749268,
                           0.6607937216758728,
                           0.6608331203460693,
                           0.6649040579795837,
                           0.6649230718612671,
                           0.6649867296218872,
                           0.6649804711341858,
                           0.6

In [26]:
record1 = Recording(dict(split_dic_decreasing_loss), dict(split_dic_decreasing_acc), dict(itrs_two), Model.GCN, Dataset.CORA, AcceptFn.INCREASING, SelectFn.NONE, Reverse.ATTACKED)
record1.save(f'data_collection/{"greedy"}_{"gcn"}_{"cora"}_{"increasing"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results({0: split_dic_decreasing_acc[0.0][0], 0.5: split_dic_decreasing_acc[0.5][0]}, ptb_rate, "Greedy", "", "increasing", "accuracy", "regular")

In [ ]:
# plot_results({0: split_dic_decreasing_loss[0.0][0], 0.5: split_dic_decreasing_loss[0.5][0]}, ptb_rate, "Greedy", "", "increasing", "loss", "regular")